<a href="https://colab.research.google.com/github/kolovratgas/HPC_labs/blob/main/lab2/Numba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numba

In [1]:
from numba import njit, jit, int32, cuda
import numba
import numpy as np
import time
from time import time
from matplotlib import pyplot as plt
import pandas as pd


%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['time', 'int32']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [46]:
x1 = 1_000
x2 = 10_000
x3 = 100_000
x4 = 500_000
x5 = 1_000_000
a1 = np.random.randint(1, 100, x1)
a2 = np.random.randint(1, 100, x2)
a3 = np.random.randint(1, 100, x3)
a4 = np.random.randint(1, 100, x4)
a5 = np.random.randint(1, 100, x5)
a_all = [a1, a2, a3, a4, a5]
c1 = np.zeros(1, dtype = np.int32)

In [47]:
threads_per_block = 32

**РАСЧЁТ НА GPU**

In [48]:
@cuda.jit
def dev_vec_sum(a, c):
  sh_b = cuda.shared.array(threads_per_block, int32)

  thread_id = cuda.threadIdx.x
  block_id = cuda.blockIdx.x
  idx = thread_id + block_id * threads_per_block
  sh_b[thread_id] = 0

  if idx < a.shape[0]:
    sh_b[thread_id] = a[idx]
    cuda.syncthreads()
    if cuda.threadIdx.x == 0:
      sum_of_arr = 0
      for i in range(threads_per_block):
        sum_of_arr += sh_b[i]
      cuda.atomic.add(c, 0, sum_of_arr)

def DeviceHost(a, c):
    dev_a = cuda.to_device(a)
    dev_c = cuda.to_device(c)
    dev_vec_sum[threads_per_block, threads_per_block](dev_a, dev_c)
    c = dev_c.copy_to_host()
    return c

In [49]:
def result_gpu(my_a):
  for i in range(len(my_a)):
    start = time.time()
    DeviceHost(my_a[i], c1)
    end = time.time() - start
    print('Расчёт на GPU: ')
    print(end)
    print(f'При размере вектора: {len(my_a[i])}')
    print('--------------------------------')

In [50]:
def cpu_vec_sum(a):
  return np.sum(a)

**РАСЧЁТ НА CPU**

In [51]:
def result_cpu(my_a):
  for i in range(len(my_a)):
    start = time.time()
    cpu_vec_sum(my_a[i])
    end = time.time() - start
    print('Расчёт на CPU: ')
    print(end)
    print(f'При размере вектора: {len(my_a[i])}')
    print('--------------------------------')

In [52]:
result_cpu(a_all)

Расчёт на CPU: 
0.0008549690246582031
При размере вектора: 1000
--------------------------------
Расчёт на CPU: 
0.0015189647674560547
При размере вектора: 10000
--------------------------------
Расчёт на CPU: 
0.00016880035400390625
При размере вектора: 100000
--------------------------------
Расчёт на CPU: 
0.011291980743408203
При размере вектора: 500000
--------------------------------
Расчёт на CPU: 
0.0010750293731689453
При размере вектора: 1000000
--------------------------------


In [53]:
result_gpu(a_all)

Расчёт на GPU: 
0.21477174758911133
При размере вектора: 1000
--------------------------------
Расчёт на GPU: 
0.005827903747558594
При размере вектора: 10000
--------------------------------
Расчёт на GPU: 
0.002980470657348633
При размере вектора: 100000
--------------------------------
Расчёт на GPU: 
0.003986358642578125
При размере вектора: 500000
--------------------------------
Расчёт на GPU: 
0.003559589385986328
При размере вектора: 1000000
--------------------------------


**РАСЧЁТ НА GPU БЕЗ УЧЁТА ПЕРЕНОСА НА ХОСТ**

In [54]:
@cuda.jit
def dev_vec_sum_new(a, c):
  sh_b = cuda.shared.array(threads_per_block, int32)

  thread_id = cuda.threadIdx.x
  block_id = cuda.blockIdx.x
  idx = thread_id + block_id * threads_per_block
  sh_b[thread_id] = 0

  if idx < a.shape[0]:
    sh_b[thread_id] = a[idx]
    cuda.syncthreads()
    if cuda.threadIdx.x == 0:
      sum_of_arr = 0
      for i in range(threads_per_block):
        sum_of_arr += sh_b[i]
      cuda.atomic.add(c, 0, sum_of_arr)

def DeviceHost_new(a, c):
    dev_a = cuda.to_device(a)
    dev_c = cuda.to_device(c)
    start1_1 = time.time()
    dev_vec_sum_new[threads_per_block, threads_per_block](dev_a, dev_c)
    end1_1 = time.time() - start1_1
    print("Расчёты на GPU без учёта копирования результатов на хост: ")
    print(end1_1)
    print(f"При размере вектора: {len(a)}")
    print('----------------------------------------------------------')
    c = dev_c.copy_to_host()

In [55]:
def result_gpu_no_hostcopy(my_a):
  for i in range(len(my_a)):
    DeviceHost_new(my_a[i], c1)

In [57]:
result_gpu_no_hostcopy(a_all)

Расчёты на GPU без учёта копирования результатов на хост: 
0.0004341602325439453
При размере вектора: 1000
----------------------------------------------------------
Расчёты на GPU без учёта копирования результатов на хост: 
0.0004918575286865234
При размере вектора: 10000
----------------------------------------------------------
Расчёты на GPU без учёта копирования результатов на хост: 
0.00042939186096191406
При размере вектора: 100000
----------------------------------------------------------
Расчёты на GPU без учёта копирования результатов на хост: 
0.00026035308837890625
При размере вектора: 500000
----------------------------------------------------------
Расчёты на GPU без учёта копирования результатов на хост: 
0.0003619194030761719
При размере вектора: 1000000
----------------------------------------------------------


**ПРОВЕРКА СООТВЕТСТВИЯ РАСЧЁТОВ**

In [58]:
def check_value(x1, x2):
  if x1 == x2:
    print(True)
  else:
    print(False)


In [59]:
check_value(DeviceHost(a1, c1), cpu_vec_sum(a1))

True


**СРАВНИТЕЛЬНАЯ ТАБЛИЦА**

In [60]:
def result_gpu_time(my_a):
  end_arr = []
  for i in range(len(my_a)):
    start = time.time()
    DeviceHost(my_a[i], c1)
    end = time.time() - start
    end_arr.append(end)
  return end_arr

r1 = result_gpu_time(a_all)

In [61]:
def result_cpu_time(my_a):
  end_arr = []
  for i in range(len(my_a)):
    start = time.time()
    cpu_vec_sum(my_a[i])
    end = time.time() - start
    end_arr.append(end)
  return end_arr

r2 = result_cpu_time(a_all)

In [62]:
@cuda.jit
def dev_vec_sum_new_time(a, c):
  sh_b = cuda.shared.array(threads_per_block, int32)

  thread_id = cuda.threadIdx.x
  block_id = cuda.blockIdx.x
  idx = thread_id + block_id * threads_per_block
  sh_b[thread_id] = 0

  if idx < a.shape[0]:
    sh_b[thread_id] = a[idx]
    cuda.syncthreads()
    if cuda.threadIdx.x == 0:
      sum_of_arr = 0
      for i in range(threads_per_block):
        sum_of_arr += sh_b[i]
      cuda.atomic.add(c, 0, sum_of_arr)

def DeviceHost_new_time(a, c):
    dev_a = cuda.to_device(a)
    dev_c = cuda.to_device(c)
    start1_1 = time.time()
    dev_vec_sum_new[threads_per_block, threads_per_block](dev_a, dev_c)
    end1_1 = time.time() - start1_1
    return end1_1  

In [63]:
def result_cpu_time_nohost(my_a):
  end_arr = []
  for i in range(len(my_a)):
    end = DeviceHost_new_time(my_a[i], c1)
    end_arr.append(end)
  return end_arr

r3 = result_cpu_time_nohost(a_all)

**ИТОГ**

In [71]:
compar_pd = pd.DataFrame({'GPU': [r1[0], r1[1], r1[2], r1[3], r1[4]], 'CPU': [r2[0], r2[1], r2[2], r2[3], r2[4]], 'GPU no host': [r3[0], r3[1], r3[2], r3[3], r3[4]] }, index = [x1, x2, x3, x4, x5] )
compar_pd

,GPU,CPU,GPU no host
1000,0.002635,0.000073,0.000832
10000,0.005284,0.000049,0.000269
100000,0.002320,0.000999,0.000299
500000,0.002014,0.002563,0.000288
1000000,0.002601,0.003093,0.000250


Итог: расчёты по GPU оказались медленнее, чем расчёты на CPU, если брать в расчёт время переноса переменной вывода с device на host. Если же не учитывать это время, то при большом размере вектора расчёты на GPU оказываются быстрее, чем расчёты на CPU.